# Emissionfactor for scope 3

### initiation

In [2]:
import boto3
import pprint
from botocore.client import Config
import json
import re

import os
import json
import time
import base64
from PIL import Image
import io
from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets
#from skimage import io



pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_runtime = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

from helper.llmcall import * 
from helper.kbhelper import * 
model = "anthropic.claude-3-sonnet"
KB_ID =  ##add KB ID creaded from last section
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"

us-west-2


### reding activities.csv - a sample BOM from ERP 

In [3]:
# Replace 'your_file.csv' with the actual path to your CSV file
import pandas as pd
df = pd.read_csv('activities.csv')
pprint.pp(df.head())

     Commodity                       CommodityDescription  \
0  67033175020                      NIPPLE BRASS 2 X 8 IN   
1  65933780021                 COUPLING SCH 40 PVC 3 SLIP   
2  67033140008         TEE BRASS 1-1/2 X 1-1/2 X 1-1/2 IN   
3  67039110016               COUPLING PVC SCHED 40 1/2 IN   
4  67033195004  COUPLING BRASS COMP GJ X COMP GJ 1-1/4 IN   

                                 ExtendedDescription  \
0  RC LN_____ QTY DEL_____ P/F_____ B/O______ DEL...   
1  RC LN_____ QTY DEL_____ P/F_____ B/O___DEL DAT...   
2  RC LN_____ QTY DEL_____ P/F_____ B/O___DEL DAT...   
3  RC LN______QTY DEL______P/F____B/O____DEL DATE...   
4  RC LN_____ QTY DEL_____ P/F_____ B/O______ DEL...   

                          ContractName  
0                  brass for Glen Bell  
1              MSC material for AWWBSC  
2               TI material for awgbsc  
3            material needed at awgbsc  
4  Brass order for Glen Bell warehouse  


In [4]:
# Prompt to clean activity names and descriptions into a simple activity description to be matched in the next step.
def generate_clean_text_prompt(COMMODITY, COMMODITY_DESCRIPTION, EXTENDED_DESCRIPTION, CONTRACT_NAME):
    
    prompt = f'''I want to do of LCA of business activities based on Environmentally Extended Input Output (EEIO) 
    Environmental Impact Factors (EIF). I am interested in the environmental impact associated with the materials 
    and manufacturing phase of the activity. I am given business activity descriptions, and I want to 
    paraphrase it to a plain language description before I select an EIF. 

    Below is an example, inside <example></example> XML tags, of a given activity, and its plain language descriptions. Note that the descriptions 
    are brief, and do not make any assumptions about the activity.

    <example>
    COMMODITY                                                      20142770002
    COMMODITY_DESCRIPTION      GLOVES WORK MECHANIC SYNTHETIC LEATHER SZ LARGE
    EXTENDED_DESCRIPTION     RC LN_____ QTY DEL_____ P/F_____ B/O______ DEL...
    CONTRACT_NAME                            MSC items for Glen Bell warehouse

    The item is a synthetic leather large work gloves 
    </example>

    Following the example, provide a plain language description of the activity data given below:
    COMMODITY              {COMMODITY}
    COMMODITY_DESCRIPTION  {COMMODITY_DESCRIPTION}
    EXTENDED_DESCRIPTION   {EXTENDED_DESCRIPTION}
    CONTRACT_NAME          {CONTRACT_NAME}

    Make the most of the given information. DO NOT say that information is limited.
    DO NOT refrain from providing a description, or ask for more information.
    If you cannot provide a plain language description, simply summarize the 
    information provided. You MUST provide a description. 

    Avoid filler words such as "Based on the details" or "happy to assist", 
    keep your response to the point.
    Do not repeat the given instructions or information. 
    DO NOT say you have insufficient information for an LCA.

    Only provide the description and nothing else.'''

    return prompt

In [5]:
for index, row in df.iterrows():
    prompt = generate_clean_text_prompt( row['Commodity'], row['CommodityDescription'], row['ExtendedDescription'], row['ContractName'])
    clean_description = interactWithLLMTextfromAnthropic(bedrock_runtime,prompt,'',model,"")
    break

In [6]:
pprint.pp(clean_description)

'The item is a 2 inch by 8 inch brass nipple.'


### search this material in the knoledge base 
### use the findings to generate potential matches of the item 

In [7]:
retrievalResults = retrieve(bedrock_agent_client,clean_description, KB_ID, 3)
contexts = get_contexts(retrievalResults)

In [8]:
pprint.pp(contexts)

['mm. or less, 1.18 inch or less), manufacturing" 332992,"Paper shells (i.e., '
 '30 mm. or less, 1.18 inch or less) manufacturing" 332992,"Pellets,',
 'mm. or less, 1.18 inch or less), manufacturing" 332992,"Paper shells (i.e., '
 '30 mm. or less, 1.18 inch or less) manufacturing" 332992,"Pellets,',
 'less, 1.18 inch or less), manufacturing" 332992,BB shot manufacturing '
 '332992,"Blank cartridges (i.e., 30 mm. or less, 1.18 inch or less) '
 'manufacturing"',
 'mm. or less, 1.18 inch or less), manufacturing" 332992,"Paper shells (i.e., '
 '30 mm. or less, 1.18 inch or less) manufacturing" 332992,"Pellets,',
 'less, 1.18 inch or less), manufacturing" 332992,BB shot manufacturing '
 '332992,"Blank cartridges (i.e., 30 mm. or less, 1.18 inch or less) '
 'manufacturing"',
 ', made from purchased metal pipe" 332996,"Nipples, metal, made from '
 'purchased pipe" 332996,Pipe and pipe fittings made from purchased metal pipe '
 '332996,Pipe couplings made from purchased metal pipe']


In [9]:
# Prompt to clean activity names and descriptions into a simple activity description to be matched in the next step.
def possible_eio_matches_system_prompt(query,search_results):
    
    prompt = f'''You are a Lifecycle Analysis expert matching business activities to their North 
    American Industry Classification System (NAICS) titles.

    I want to do of LCA of business activities based on Environmentally Extended Input Output 
    (EEIO) Environmental Impact Factors (EIF). I am interested in the environmental
    impact associated with the materials and manufacturing phase of the activity. 
    I am given a business activity and I want to match it to its the North
    American Industry Classification System code and title.

    Format the output in JSON with the keys "NAICSCode1", "NAICSTitle1", "NAICSCode2", "NAICSTitle2", "NAICSCode3", "NAICSTitle3".

    Below is an example, inside <example></example> XML tags, of a given activity and three possible NAICS codes and titles.

    <example>
    The item is a synthetic leather large work gloves.

    {{
        "NAICSCode1": "315990",
        "NAICSTitle1": "Apparel Accessories and Other Apparel Manufacturing",
        "NAICSCode2": "339920",
        "NAICSTitle2": "Sporting and Athletic Goods Manufacturing",
        "NAICSCode3": "316998",
        "NAICSTitle3": "All Other Leather Good and Allied Product Manufacturing"
    }}
    </example>

    What are three possible NAICS titles for the given activity:
    {query}

    {search_results}

    Make the most of the given information. DO NOT say that information is limited or ask for more information.
    YOU MUST provide three NAICS titles.
    Avoid filler words such as "Based on the details" or "happy to assist", keep your response to the point.
    Do not repeat the given instructions or information. 
    DO NOT say you have insufficient information for an LCA.

    Respond with the JSON output and nothing else.
    '''
    #print(prompt)
    return prompt

In [10]:
prompt = possible_eio_matches_system_prompt(clean_description, contexts)
possible_match = interactWithLLMTextfromAnthropic(bedrock_runtime,prompt,'',model,"")
    

In [11]:
pprint.pp(possible_match)

('{\n'
 '    "NAICSCode1": "332996",\n'
 '    "NAICSTitle1": "Fabricated Pipe and Pipe Fitting Manufacturing",\n'
 '    "NAICSCode2": "331491",\n'
 '    "NAICSTitle2": "Nonferrous Metal (except Copper and Aluminum) Rolling, '
 'Drawing, and Extruding",\n'
 '    "NAICSCode3": "332919",\n'
 '    "NAICSTitle3": "Other Metal Valve and Pipe Fitting Manufacturing"\n'
 '}')


In [12]:
# Parse the JSON string
data = json.loads(possible_match)

# Prepare and format the output
output = []
for i in range(1, 4):
    code = data[f'NAICSCode{i}']
    title = data[f'NAICSTitle{i}']
    output.append(f'{code} - {title}')

# Join the output with new lines
formatted_input = '\n'.join(output)



In [13]:
pprint.pp(formatted_input)

('332996 - Fabricated Pipe and Pipe Fitting Manufacturing\n'
 '331491 - Nonferrous Metal (except Copper and Aluminum) Rolling, Drawing, and '
 'Extruding\n'
 '332919 - Other Metal Valve and Pipe Fitting Manufacturing')


### find the best match 

In [14]:
# Prompt to choose the best emission factor from the options
def best_eif_prompt(query,formatted_input):
    
    best_eif_prompt = f'''You are a Lifecycle Analysis expert matching business activities to their North American Industry Classification System (NAICS) titles.

    I want to do of LCA of business activities based on Environmentally Extended Input Output (EEIO) Environmental Impact Factors (EIF). I am interested in the environmental impact associated with the materials and manufacturing phase of the activity. I am given a business activity and three possible corresponding NAICS codes and titles. 

    I want to pick the NAICS code and title that best match the given activity. Include justification for your choice.
    Format the output in JSON with the keys "BestNAICSCode", "BestNAICSTitle", "Justification".

    Activity:
    {query}

    Possible NAICS codes and titles:
    {formatted_input}

    Which of these impact factors is the best match for the provided activity? 

    Note that impact factor names with 'market' in them are better match than those with 'production' in them.
    Make the most of the given information. DO NOT say that information is limited or ask for more information.
    YOU MUST choose a best code and title. YOU MUST include a justification for your choice.
    Avoid filler words such as "Based on the details" or "happy to assist", keep your response to the point.
    Do not repeat the given instructions or information. 
    DO NOT say you have insufficient information for an LCA.

    Respond with the JSON output and nothing else.
    '''
    return best_eif_prompt

In [15]:
prompt = best_eif_prompt(clean_description,formatted_input)
best_match = interactWithLLMTextfromAnthropic(bedrock_runtime,prompt,'',model,"")
    

In [16]:
pprint.pp(best_match)

('{"BestNAICSCode": "332919", "BestNAICSTitle": "Other Metal Valve and Pipe '
 'Fitting Manufacturing", "Justification": "The activity involves '
 'manufacturing a brass pipe fitting, specifically a 2 inch by 8 inch brass '
 "nipple. The NAICS code 332919 'Other Metal Valve and Pipe Fitting "
 "Manufacturing' directly aligns with this activity, as it covers the "
 'manufacturing of metal pipe fittings like the brass nipple mentioned."}')


In [17]:
# Parse the JSON string
data = json.loads(best_match)

# Prepare and format the output

bestcode = data[f'BestNAICSCode']
besttitle = data[f'BestNAICSTitle']
justification = data[f'Justification']
    



### retrive the emmision factor of the best match NAICS code 

In [18]:
def get_contextswithretriveandgenerate(retrievalResults):
    data = json.loads(json.dumps(retrievalResults))
    # Extract the output text value
    output_text = data['output']['text']
    return output_text


In [19]:
question = "what is the Supply Chain Emission Factors with Margins for " + besttitle +". only return the numerical value of the measurement."
retrievalResults = retrieveandgenerate(bedrock_agent_client,question, KB_ID,model_arn)
#print(retrievalResults)
emmision_fector = get_contextswithretriveandgenerate(retrievalResults)
#print(emmision_fector)

In [20]:
pprint.pp(emmision_fector)

'0.171'
